In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break_times = []
opr_2_break_times = []
shift_started = True
end = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request() as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))

In [6]:
def is_available(opr):
    return opr.count == 0

In [7]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [8]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [9]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [10]:
def shift(env):
    shift_started = False
    if not end:
        [opr_1_break, opr_2_break] = break_number()
        print(opr_1_break)
        print(opr_2_break)
        break_time(opr_1_break, opr_2_break)
        for i in opr_1_break_times:
            env.process(take_a_break(env, i, operator1, shift_started))
        for i in opr_2_break_times:
            env.process(take_a_break(env, i, operator2, shift_started))
        yield env.timeout(480)
        shift_started = True
        env.process(shift(env))
        print("the time %g" % env.now)
        opr_1_break_times.clear()
        opr_2_break_times.clear()

In [11]:
def take_a_break(env, number, opr, shift_started):
    yield env.timeout(number)
    with opr.request() as brk:
        yield brk
        if not shift_started:
            print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
            yield env.timeout(3)

In [12]:
def customer_generator(env):
    global end
    start = True
    for i in range(1000):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
    end = True

In [13]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

5
6
[44.63307524333047, 74.39519318760645, 209.12059240981336, 255.9664319411964, 385.08427703724]
[34.07604646614267, 226.6161423659573, 292.4094909717117, 387.8161389706391, 397.1018255104171, 414.16006451794135]
Customer 1 initiated a call at 26.5136
Customer 2 initiated a call at 27.7602
Customer 3 initiated a call at 29.5965
Customer 2 is rooted to operator 2 at 30.0202
Customer 2 is assigned to the operator 2 at 30.0202
Customer 2 is done with operator 2 at 36.1137
break started for operator 2 at 36.1137
Customer 3 is rooted to operator 2 at 36.1207
Customer 1 is rooted to operator 1 at 38.2832
Customer 1 is assigned to the operator 1 at 38.2832
Customer 3 is assigned to the operator 2 at 39.1137
Customer 4 initiated a call at 39.8863
Customer 5 initiated a call at 41.1031
Customer 5 is rooted to operator 2 at 41.2097
Customer 3 is done with operator 2 at 45.6809
Customer 5 is assigned to the operator 2 at 45.6809
Customer 6 initiated a call at 45.8921
Customer 4 hangs up the cal

Customer 360 initiated a call at 2094.58
Customer 356 is done with operator 2 at 2094.84
Customer 359 is assigned to the operator 2 at 2094.84
Customer 360 is rooted to operator 2 at 2100.32
Customer 359 is done with operator 2 at 2100.43
Customer 358 is assigned to the operator 2 at 2100.43
Customer 358 is done with operator 2 at 2105.61
Customer 360 is assigned to the operator 2 at 2105.61
Customer 361 initiated a call at 2107.44
Customer 360 is done with operator 2 at 2111.28
break started for operator 2 at 2111.28
Customer 362 initiated a call at 2111.55
Customer 362 is rooted to operator 2 at 2113.89
break started for operator 2 at 2114.28
Customer 361 is rooted to operator 2 at 2115.54
Customer 362 is assigned to the operator 2 at 2117.28
Customer 362 is done with operator 2 at 2120.16
Customer 361 is assigned to the operator 2 at 2120.16
Customer 363 initiated a call at 2121.2
Customer 361 is done with operator 2 at 2122.43
Customer 364 initiated a call at 2122.64
Customer 364 i

Customer 612 is rooted to operator 2 at 3569.43
Customer 611 is done with operator 2 at 3572.15
Customer 612 is assigned to the operator 2 at 3572.15
break started for operator 1 at 3573.9
Customer 612 is done with operator 2 at 3573.91
Customer 613 is rooted to operator 1 at 3575.02
Customer 614 initiated a call at 3575.44
Customer 614 is rooted to operator 2 at 3575.94
Customer 614 is assigned to the operator 2 at 3575.94
Customer 613 is assigned to the operator 1 at 3576.9
Customer 614 is done with operator 2 at 3581.49
Customer 613 is done with operator 1 at 3589.3
Customer 615 initiated a call at 3592.26
Customer 615 is rooted to operator 1 at 3597.49
Customer 615 is assigned to the operator 1 at 3597.49
Customer 616 initiated a call at 3599.56
Customer 617 initiated a call at 3601.83
Customer 618 initiated a call at 3604.67
Customer 615 is done with operator 1 at 3607.27
Customer 619 initiated a call at 3608.47
Customer 619 is rooted to operator 2 at 3608.56
Customer 619 is assig

Customer 843 is rooted to operator 2 at 4855.61
Customer 843 is assigned to the operator 2 at 4855.61
Customer 842 is rooted to operator 1 at 4857.19
Customer 842 is assigned to the operator 1 at 4857.19
Customer 843 is done with operator 2 at 4859.08
Customer 844 initiated a call at 4861.72
Customer 844 is rooted to operator 1 at 4863.55
Customer 842 is done with operator 1 at 4870.21
Customer 844 is assigned to the operator 1 at 4870.21
Customer 845 initiated a call at 4879.63
Customer 844 is done with operator 1 at 4881.02
break started for operator 1 at 4881.02
break started for operator 2 at 4881.85
Customer 845 hangs up the call at 4883.02
Customer 846 initiated a call at 4889.38
Customer 847 initiated a call at 4890.2
Customer 847 is rooted to operator 1 at 4892.52
Customer 847 is assigned to the operator 1 at 4892.52
Customer 846 is rooted to operator 2 at 4895.54
Customer 846 is assigned to the operator 2 at 4895.54
Customer 846 is done with operator 2 at 4900.95
Customer 847 